<a href="https://colab.research.google.com/github/eduardoplima/artists-expenditure-llm/blob/main/artists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identifying artists in public expenditure using LLMs

## Author: Eduardo P. Lima

## Summary

The Brazilian Audit Courts have, among their constitutional attributions, the responsibility of monitoring the expenses with cultural events and artistic presentations in general of the government departments that report to them. To this end, the Audit Courts receive information from the departments under their jurisdiction about the expenditures of this nature.

However, this information is not structured in a way that facilitates the identification of the artists hired. Therefore, it is necessary to use Natural Language Processing techniques to extract this information in order to assess the regular payment of these contracts.

This notebook shows the use of techniques for this purpose, especially the use of Large Language Models (LLM).

### Keypoints

* Point 1




In [18]:
!pip install gdown langchain_openai langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.4 MB/s eta 0:00:00


In [20]:
import os
import gdown
import getpass

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


from langgraph.graph import StateGraph, END
#from langchain_openai import

## Dataset loading

We load our dataset from the xlsx file. It has 3 columns, concerning the description of the procurement process, contract and subsequent prepayment. We have to look for an artist identification in those columns.

In [9]:
url = "https://github.com/eduardoplima/artists-expenditure-llm/raw/refs/heads/main/festas_juninas.xlsx"
output = "artists.xlsx"
gdown.download(url, output)

Downloading...
From: https://github.com/eduardoplima/artists-expenditure-llm/raw/refs/heads/main/festas_juninas.xlsx
To: /content/artists.xlsx
100%|██████████| 1.14M/1.14M [00:00<00:00, 84.9MB/s]


'artists.xlsx'

In [10]:
#df_art = pd.read_csv('artists.csv', on_bad_lines='skip')
df_art = pd.read_excel('artists.xlsx', engine='openpyxl')

In [11]:
df_art.head(10)

,contract,prepayment,procurement
0,contratação da empresa A. NUNES DE ARAÚJO PROD...,"Despesa com diária em favor da servidora, NAYA...",Contratação de empresa especializada no fornec...
1,contratação da empresa A. NUNES DE ARAÚJO PROD...,Ref. empenho estimativo de diárias nacionais p...,Contratação de empresa especializada no fornec...
2,contratação da empresa A. NUNES DE ARAÚJO PROD...,Ref. empenho estimativo de diárias internacion...,Contratação de empresa especializada no fornec...
3,NaN,Referente despesa com 4º termo aditivo empenho...,NaN
4,contratação da empresa A. NUNES DE ARAÚJO PROD...,Referente despesa do 4º termo aditivo empenho ...,Contratação de empresa especializada no fornec...
5,NaN,Ref. serviço de fornecimento de passagens aére...,NaN
6,NaN,Ref. serviço de fornecimento de passagens aére...,NaN
7,NaN,Referente despesa com participação no lounge m...,NaN
8,contratação da empresa A. NUNES DE ARAÚJO PROD...,Referente empenho com participação no evento s...,Contratação de empresa especializada no fornec...
9,NaN,Despesa com participação Expoturismo Paraná d...,NaN


## Model creation

We create the models that we'll use on our agents

In [15]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


In [17]:
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")